In [9]:
import pandas as pd
import os

def generate_track_pages(csv_path, output_dir='output'):
  
    try:
       
        df = pd.read_csv(csv_path)
        
        
        required_columns = ['title', 'abstract', 'authors', 'track', 'invited','transaction']
        if not all(col in df.columns for col in required_columns):
            missing = [col for col in required_columns if col not in df.columns]
            raise ValueError(f"CSV缺少必要列：{missing}")

        os.makedirs(output_dir, exist_ok=True)

        
        track_files = []
        for track_name, group in df.groupby('track'):
            
            safe_name = sanitize_filename(track_name)
            filename = os.path.join(output_dir, f"{safe_name}.html")
            
            
            html_content = generate_single_track_html(track_name, group)
            
           
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(html_content)
            track_files.append((track_name, filename))

        
        generate_index_page(track_files, output_dir)
        
        print(f"Success, at{os.path.abspath(output_dir)}")

    except Exception as e:
        print(f"Fail：{str(e)}")

def sanitize_filename(name):
    
    return "".join([c if c.isalnum() or c in (' ', '_') else '_' for c in name]).strip()

def generate_single_track_html(track_name, df):
    
    papers_html = []
    for _, row in df.iterrows():
        invited_tag = '<span class="invited">[Invited]</span>' if row['invited'] else ''
        papers_html.append(f"""
        <div class="paper">
            <h3 class="title">{row['title']}{invited_tag}</h3>
            <div class="authors">{row['authors']}</div>
            <div class="abstract">{row['abstract']}</div>
            <a href={"..\\pdf\\"+row['transaction']+"-final.pdf"}>pdf</a>
            
        </div>
        """)
    
    return f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>{track_name}</title>
    <style>
        body {{ font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }}
        h1 {{ color: #2c3e50; border-bottom: 2px solid #3498db; }}
        .paper {{ margin: 20px 0; padding: 15px; border-left: 4px solid #3498db; }}
        .title {{ font-weight: bold; color: #2c3e50; }}
        .authors {{ color: #7f8c8d; font-size: 0.9em;font-style: italic; }}
        .abstract {{ color: #666; margin-top: 15px; font-size: 0.95em; }}
        .invited {{ color: #27ae60; font-weight: bold; }}
    </style>
</head>
<body>
    <h1>{track_name} </h1>
    {"".join(papers_html)}
    <p><a href="index.html">← Back</a></p>
</body>
</html>"""

def generate_index_page(track_files, output_dir):
    
    links = []
    for name, path in track_files:
        filename = os.path.basename(path)
        links.append(f'<li><a href="{filename}">{name}</a></li>')
    
    index_html = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Tracks</title>
    <style>
        body {{ font-family: Arial, sans-serif; max-width: 600px; margin: 0 auto; padding: 20px; }}
        h1 {{ text-align: center; color: #2c3e50; }}
        ul {{ list-style: none; padding: 0; }}
        li {{ margin: 10px 0; padding: 10px; background: #f8f9fa; border-radius: 4px; }}
        a {{ color: #3498db; text-decoration: none; }}
        a:hover {{ text-decoration: underline; }}
    </style>
</head>
<body>
    <h1>EDTM 2025 Papers</h1>
    <ul>{"".join(links)}</ul>
</body>
</html>"""

    with open(os.path.join(output_dir, 'index.html'), 'w', encoding='utf-8') as f:
        f.write(index_html)


if __name__ == "__main__":
    csv_path = "accepted_paper_list.csv"
    generate_track_pages(csv_path)


Success, ate:\The University Of Hong Kong\CANLab - WEN, Bo - WEN, Bo\EDTM\EDTM_proceeding\output
